Installing the PRAW library

In [1]:
!pip install praw transformers accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.5 MB/s eta 0:00:00


Authenticating using PRAW

In [2]:
import praw

reddit = praw.Reddit(
    client_id='FpU5eDyJ2LUlko3OsyuvqQ',
    client_secret='0EHBx5aGIQgjaufl0Fmfl-CtghJdyw',
    user_agent='UserPersonaScraper by /u/monokrrome',
    username='monokrrome',
    password='joonjoon0.',
    check_for_async=False
)

Scrape a user's posts and comments

In [3]:
def scrape_user_data(username, post_limit=20, comment_limit=20):
    redditor = reddit.redditor(username)
    data = {"posts": [], "comments": []}

    for post in redditor.submissions.new(limit=post_limit):
        data["posts"].append({
            "title": post.title,
            "text": post.selftext,
            "url": post.url,
            "subreddit": str(post.subreddit)
        })

    for comment in redditor.comments.new(limit=comment_limit):
        data["comments"].append({
            "text": comment.body,
            "url": f"https://www.reddit.com{comment.permalink}",
            "subreddit": str(comment.subreddit)
        })

    return data

Format data into prompt

In [4]:
def format_prompt(user_data):
    examples = []
    for p in user_data['posts']:
        examples.append(f"Post: {p['title']}\n{p['text']}\nURL: {p['url']}")
    for c in user_data['comments']:
        examples.append(f"Comment: {c['text']}\nURL: {c['url']}")
    content = "\n\n".join(examples[:10])
    return f"""
You are an expert in digital behavior analysis.
Below are Reddit posts and comments from a single user.
Generate a persona covering these traits: Interests, Tone, Values, Personality, Motivations, Frustrations, Goals, and Needs.
For each trait, cite the post or comment that supports your conclusion.

{content}

---Output Format---
User Persona:
- Interests: ... [Source: "Comment/Post..."]
- Tone: ...
- Values: ...
- Personality: ...
- Behavior: ...
- Frustrations: ...
- Motivations: ...
- Goals: ...
- Needs: ...
"""

Load openchat/openchat-3.5-0106 Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-0106")
model = AutoModelForCausalLM.from_pretrained("openchat/openchat-3.5-0106", device_map="auto")
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

Generate Persona

In [ ]:
def generate_persona(prompt):
    output = llm(prompt, max_new_tokens=1024, temperature=0.7)
    full_text = output[0]["generated_text"]
    return full_text[len(prompt):].strip()

Main Execution

In [ ]:
username = "kojied"  # or any Reddit username
user_data = scrape_user_data(username)
prompt = format_prompt(user_data)
persona = generate_persona(prompt)

Save Output

In [ ]:
with open(f"{username}_persona.txt", "w") as f:
    f.write(persona)

print("Persona saved as", f"{username}_persona.txt")
print("\n Generated Persona:\n")
print(persona)